***
# importとdef

In [ ]:
from pandas import json_normalize
import pandas as pd
import json
import datetime
import glob
import re

In [ ]:
def get_files(date):
    # ファイルパスのパターン定義
    pattern = f"/Users/name/vscode/task/dynamic_pricing-main/bleague_scraping/ryukyu/data/*_{date}_price.json" # *_{date}_price.jsonが格納されているパスを入力する
    
    # パターンに一致するファイルのリストを取得
    files = glob.glob(pattern)
    
    # gamedateの要素を抽出するためのリストを初期化
    gamedates = []
    
    # ファイル名からgamedateを抽出
    for file in files:
        match = re.search(r"(\d+)_\d{4}_price\.json", file)
        if match:
            gamedates.append(match.group(1))     
    
    for gamedate in gamedates:
        #priceに関するjsonデータを取得
        filepath_price = f"/Users/name/vscode/task/main/bleague_scraping/ryukyu/data/{gamedate}_{date}_price.json" # *_{date}_price.jsonが格納されているパスを入力する
        #sectionに関するjsonデータを取得
        filepath_section = f"/Users/name/vscode/task/main/bleague_scraping/ryukyu/data/{gamedate}_{date}_section.json" # *_{date}_section.jsonが格納されているパスを入力する


        # JSONデータを変換
        data = pd.read_json(filepath_section)
        # sectionデータからセクション名、priceLevel（sectionsデータとの紐付け用ID）、amount（在庫数）を抽出
        results = []
        for section, details in data.items():
            for price_info in details["availability"]["prices"]:
                result = {
                    "section": section,
                    "priceLevel": price_info["priceLevel"],
                    "amount": price_info["amount"]
                    }
                results.append(result)
            section_data = pd.DataFrame(results) # section_dataが完成


        # ファイルを開き、JSONとして読み込む
        with open(filepath_price, 'r', encoding='utf-8') as file:
            data = json.load(file)
        # priceデータからgame_info（試合日を抜き出す）、game_info2（awayチームを抜き出す）、priceLevelID（priceデータとの紐付け用ID）、label（席種名）、priveTypeID（販売者区分）、base（値段）を抽出
        price_deta = []
        for offer in data['offerPrices']:
            for zone in offer['zonePrices']:
                for level in zone['priceLevels']:
                    for price in level['prices']:
                        price_deta.append({
                            'game_info' : offer['offerLookupId'],
                            'game_info2' : offer["offerName"],
                            'priceLevelID': level['priceLevelID'],
                            'label': level['label'],
                            'priceTypeID': price['priceTypeID'],
                            'base': price['base']
                            })
        price_data = pd.DataFrame(price_deta) # price_dataが完成


        # データの取得日をdata_get_dateに記入
        year_string = datetime.datetime.now().year
        formatted_date_string = f"{year_string}{date}"
        price_data["data_get_date"] = pd.to_datetime(formatted_date_string,format='%Y%m%d')

        # game_info内の連続する数値8つを試合日としgame_dateに記入
        price_data['game_date'] = price_data['game_info'].str.extract(r'(\d{8})')
        price_data['game_date'] = pd.to_datetime(price_data['game_date'], format='%Y%m%d')

        # ホームチームをhome_teamに記入
        price_data["home_team"] = "琉球"
        
        # game_info内のvsの後の文字をアウェイチームとしaway_teamに記入
        price_data['away_team'] = price_data['game_info2'].str.extract(r'vs([^\s]+)')
        
        # priceTypeID（販売者区分）から販売者の情報を引き出しpriveTyepIDに記入
        price_data["priceTypeID"].replace({
            "650": "大人・こども共通",
            "651": "大人",
            "652": "こども"
            }, inplace=True)


        # 仕上げ処理
        try:
            # priceLevelとpriceLevelIDを紐付け、price_dataとsection_dataを一つに結合
            pl = pd.merge(section_data, price_data, left_on='priceLevel', right_on='priceLevelID', how='inner')

            # 100倍になっている値段を修正
            pl["base"] = pl["base"] / 100

            # カラム名を分かりやすい名前に修正
            pl = pl.rename(columns={
                "priceLevel" : "席種ID",
                "amount" : "残り席数",
                "label" : "席種名",
                "priceTypeID" : "販売者区分",
                "base" : "値段"
                })
            
            # 必要ないカラムを削除
            pl = pl.drop(columns=["席種ID","game_info","game_info2","priceLevelID"])
            
            # ファイル名を「試合日_データ取得日_ホームチーム名.csv」となるように設定し保存する
            data_get_date_filename, game_date_filename = pl["data_get_date"][0].strftime("%m%d"), pl["game_date"][0].strftime("%m%d")
            pl.to_csv(f"{game_date_filename}_{data_get_date_filename}_ryukyu.csv",encoding="shift-jis",index=False)
            
        except KeyError:
            pass

***
# 本日分のデータを加工するコード

In [ ]:
#今日の日付を4桁の数字で取得
date = pd.to_datetime(datetime.date.today()).strftime("%m%d")

In [ ]:
get_files(date)

***
# 以前の日付のデータを加工するコード

In [ ]:
#今日の日付を4桁の数字で取得
date_you_use = "0307" # 任意の日付を入力する

In [ ]:
get_files(date)